In [ ]:
# ==============================================================================
# AI-15: UNIVERSAL CLUSTERING TOOLKIT (FINAL VERSION 3.0)
# Fitur: Auto-Adapter, RFM Analysis, Auto-Labeling, & Real-Value Visualization
# ==============================================================================

import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import warnings

# Matikan peringatan yang mengganggu
warnings.filterwarnings('ignore')
sns.set(style="whitegrid", context="notebook")

class UniversalToolkit:
    def __init__(self, filepath):
        self.filepath = filepath
        self.filename = filepath.split('/')[-1] # Ambil nama file saja
        self.data_raw = None
        self.data_processed = None # Data untuk clustering (RFM / Features)
        self.data_original_values = None # Data untuk visualisasi (Angka asli)
        self.mode = "UNKNOWN"
        
    def load_and_adapt(self):
        print(f"\n{'='*60}")
        print(f"🚀 MEMPROSES FILE: {self.filename}")
        
        # 1. Load Data
        try:
            df = pd.read_csv(self.filepath, encoding='ISO-8859-1')
        except:
            df = pd.read_csv(self.filepath)
            
        # 2. AUTO-ADAPTER (Penerjemah Kolom Otomatis)
        cols = [c.lower() for c in df.columns]
        
        # Skenario A: Retail Sales Dataset (Transaction ID, Date, Total Amount)
        if 'transaction id' in cols or 'transaction_id' in cols:
            print("   -> Terdeteksi: Format Retail Sales")
            mapper = {
                'Transaction ID': 'InvoiceNo', 'Date': 'InvoiceDate', 
                'Customer ID': 'CustomerID', 'Total Amount': 'TotalPrice',
                'Price per Unit': 'UnitPrice', 'Product Category': 'Category'
            }
            # Rename kolom yang ada di mapper
            df.rename(columns={k: v for k, v in mapper.items() if k in df.columns}, inplace=True)
            
        # Skenario B: Customer Shopping Data (invoice_no, invoice_date, price)
        elif 'invoice_no' in cols:
            print("   -> Terdeteksi: Format Customer Shopping")
            mapper = {
                'invoice_no': 'InvoiceNo', 'invoice_date': 'InvoiceDate', 
                'customer_id': 'CustomerID', 'price': 'UnitPrice', 
                'quantity': 'Quantity'
            }
            df.rename(columns={k: v for k, v in mapper.items() if k in df.columns}, inplace=True)
            
        # Skenario C: Marketing Data (Ad Group, Impressions)
        elif 'ad group' in cols or 'campaign' in cols:
            print("   -> Terdeteksi: Data Marketing/Iklan")
            self.mode = "MARKETING"
            # Buat ID palsu untuk keperluan sistem
            if 'Ad Group' in df.columns: df['Ref_ID'] = df['Ad Group']
            else: df['Ref_ID'] = df.index
            
        self.data_raw = df
        return df

    def preprocess(self):
        df = self.data_raw.copy()
        
        # --- MODE 1: TRANSAKSI (E-COMMERCE) ---
        # Ciri: Ada kolom InvoiceDate
        if 'InvoiceDate' in df.columns:
            self.mode = "TRANSACTION (RFM)"
            print("   -> Mode Analisis: RFM (Recency, Frequency, Monetary)")
            
            # Cleaning & Parsing Date
            df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], dayfirst=True, format='mixed', errors='coerce')
            df = df.dropna(subset=['InvoiceDate'])
            
            # Pastikan ada TotalPrice
            if 'TotalPrice' not in df.columns:
                if 'Quantity' in df.columns and 'UnitPrice' in df.columns:
                    df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
                else:
                    print("   ❌ Error: Tidak bisa hitung Total Belanja. Kolom kurang.")
                    return False

            # Hitung RFM
            latest_date = df['InvoiceDate'].max() + dt.timedelta(days=1)
            rfm = df.groupby('CustomerID').agg({
                'InvoiceDate': lambda x: (latest_date - x.max()).days,
                'InvoiceNo': 'count',
                'TotalPrice': 'sum'
            })
            rfm.rename(columns={'InvoiceDate': 'Recency', 'InvoiceNo': 'Frequency', 'TotalPrice': 'Monetary'}, inplace=True)
            
            # Simpan data asli untuk visualisasi nanti
            self.data_original_values = rfm.copy()
            # Simpan data untuk clustering (akan di-scale nanti)
            self.data_processed = rfm
            
        # --- MODE 2: MARKETING (IKLAN) ---
        elif self.mode == "MARKETING":
            print("   -> Mode Analisis: Performance Clustering")
            # Ambil kolom metrik penting
            metrics = ['Impressions', 'Clicks', 'Cost', 'Revenue', 'Conversions']
            available_metrics = [c for c in metrics if c in df.columns]
            
            self.data_original_values = df.set_index('Ref_ID')[available_metrics].fillna(0)
            self.data_processed = self.data_original_values.copy()
            
        # --- MODE 3: FALLBACK ---
        else:
            print("   -> Mode Analisis: Generic Profiling")
            self.data_original_values = df.select_dtypes(include=[np.number]).fillna(0)
            self.data_processed = self.data_original_values.copy()
            
        return True

    def run_clustering(self, k=3):
        # 1. Scaling (Wajib agar K-Means akurat)
        scaler = StandardScaler()
        data_scaled = scaler.fit_transform(self.data_processed)
        
        # 2. K-Means
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        labels = kmeans.fit_predict(data_scaled)
        
        # Masukkan label ke data asli (agar bisa dibaca manusia)
        self.data_original_values['Cluster_Label'] = labels
        
        # 3. Auto-Naming (Memberi Nama Segmen)
        self._assign_business_labels()
        
        # 4. Validasi Silhouette
        try:
            score = silhouette_score(data_scaled, labels)
            print(f"   ✅ Clustering Selesai. Silhouette Score: {score:.3f}")
        except:
            pass

    def _assign_business_labels(self):
        df = self.data_original_values
        
        # Jika Mode RFM: Label berdasarkan Uang (Monetary)
        if "Monetary" in df.columns:
            # Urutkan cluster berdasarkan rata-rata belanja terbesar
            means = df.groupby('Cluster_Label')['Monetary'].mean().sort_values(ascending=False)
            
            # Mapping Nama
            label_map = {}
            names = ['Platinum (VIP)', 'Gold (Loyal)', 'Silver (Standard)', 'Bronze (Low)']
            for i, cluster_id in enumerate(means.index):
                if i < len(names): label_map[cluster_id] = names[i]
                else: label_map[cluster_id] = f'Segment {i+1}'
                
            df['Segment_Name'] = df['Cluster_Label'].map(label_map)
            
        # Jika Mode Marketing: Label berdasarkan Revenue/Clicks
        elif "Revenue" in df.columns or "Clicks" in df.columns:
            target = 'Revenue' if 'Revenue' in df.columns else 'Clicks'
            means = df.groupby('Cluster_Label')[target].mean().sort_values(ascending=False)
            
            label_map = {}
            names = ['Winning Ads (Top)', 'Potential', 'Underperforming']
            for i, cluster_id in enumerate(means.index):
                if i < len(names): label_map[cluster_id] = names[i]
                else: label_map[cluster_id] = f'Cluster {i}'
                
            df['Segment_Name'] = df['Cluster_Label'].map(label_map)
            
        else:
            df['Segment_Name'] = "Cluster " + df['Cluster_Label'].astype(str)

    def visualize(self):
        print("   🎨 Membuat Visualisasi (Data Asli)...")
        df = self.data_original_values
        
        # Tentukan Sumbu X dan Y
        if "Recency" in df.columns:
            x, y = 'Recency', 'Monetary'
            title = "Peta Pelanggan: Seberapa Baru vs Seberapa Banyak Belanja"
        elif "Cost" in df.columns:
            x = 'Cost'
            y = 'Revenue' if 'Revenue' in df.columns else 'Clicks'
            title = "Peta Iklan: Biaya vs Hasil"
        else:
            cols = df.select_dtypes(include=[np.number]).columns
            x, y = cols[0], cols[1]
            title = "Clustering Map"
            
        plt.figure(figsize=(10, 6))
        
        # Plot dengan Data ASLI (Bukan Scaled)
        sns.scatterplot(data=df, x=x, y=y, hue='Segment_Name', palette='viridis', s=100, alpha=0.8)
        
        plt.title(f"{title} ({self.filename})")
        
        # Pakai Log Scale jika angka uangnya jomplang (Jutaan vs Ribuan)
        if y in ['Monetary', 'Revenue', 'Total Amount'] and df[y].min() > 0:
            plt.yscale('log')
            plt.ylabel(f"{y} (Log Scale)")
        else:
            plt.ylabel(y)
            
        plt.xlabel(x)
        plt.legend(title='Segmen Bisnis', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        plt.show()
        
        # Simpan Hasil
        output_file = f"Hasil_{self.filename}"
        df.to_csv(output_file)
        print(f"   💾 Hasil tersimpan di: {output_file}")
        
        # Tampilkan Preview Rata-Rata
        print("\n   📊 PROFIL SEGMEN (Rata-rata):")
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        if 'Cluster_Label' in numeric_cols: numeric_cols.remove('Cluster_Label')
        display(df.groupby('Segment_Name')[numeric_cols].mean().round(2))

# ==============================================================================
# MAIN PROGRAM (JALANKAN UNTUK 3 FILE ANDA)
# ==============================================================================

# Daftar file yang Anda punya
files = [
    'retail_sales_dataset.csv', 
    'customer_shopping_data.csv', 
    'final_shop_6modata.csv'
]

# Loop untuk memproses setiap file
for file in files:
    toolkit = UniversalToolkit(file)
    toolkit.load_and_adapt() # 1. Load & Adaptasi Kolom
    
    if toolkit.preprocess(): # 2. Cek apakah data valid
        toolkit.run_clustering(k=3) # 3. Cluster
        toolkit.visualize() # 4. Gambar Grafik